In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta

In [2]:
stats_equipos = pd.read_csv("./input.csv")
distancias = pd.read_csv("./distancias.csv")
stats_equipos = stats_equipos.drop_duplicates(['ID Partido'], keep='last')

In [3]:
stats_equipos['number_date'] = stats_equipos['ID Partido'].replace('[A-z]', '', regex = True)
stats_equipos['number_date'] = stats_equipos['number_date'].replace('\.', '', regex = True)
stats_equipos['number_date'] = stats_equipos['number_date'].replace('\/', '', regex = True)
stats_equipos['number_date'] = stats_equipos['number_date'].str.slice(0,8)

In [4]:
stats_equipos['number_date'] = pd.to_datetime(stats_equipos['number_date'], format='%Y%m%d', errors='ignore')

In [5]:
stats_equipos['previous_date'] = stats_equipos['number_date'] - timedelta(days =1)
stats_equipos['previous_date'] = pd.to_datetime(stats_equipos['previous_date'], format='%Y%m%d', errors='ignore')

In [6]:
stats_equipos['local_dif_between_previous_game'] = 0
stats_equipos['visitor_dif_between_previous_game'] = 0
stats_equipos['local_dif_between_previous_game'] = pd.to_datetime(stats_equipos['local_dif_between_previous_game'], format='%Y%m%d', errors='ignore')
stats_equipos['visitor_dif_between_previous_game'] = pd.to_datetime(stats_equipos['visitor_dif_between_previous_game'], format='%Y%m%d', errors='ignore')

In [7]:
stats_equipos['local_played_previous_date'] = 0
stats_equipos['visitor_played_previous_date'] = 0
stats_equipos['previous_10_days'] = stats_equipos['number_date'] - timedelta(days =10)
stats_equipos['previous_10_days'] = pd.to_datetime(stats_equipos['previous_10_days'], format='%Y%m%d', errors='ignore')
stats_equipos['local_played_local'] = 0
stats_equipos['local_played_visitor'] = 0
stats_equipos['visitor_played_visitor'] = 0
stats_equipos['visitor_played_local'] = 0
stats_equipos ['place_local_played'] = 'Nada'
stats_equipos ['place_visitor_played'] = 'Nada'
stats_equipos ['avg_distance_place_visitor_played'] = -1
stats_equipos ['avg_distance_place_local_played'] = -1


for actual in stats_equipos.index:
    
    jugo_dia_anterior = stats_equipos[(stats_equipos ['number_date'] == stats_equipos.loc[actual,'previous_date'])]
    
    visitor_jugo_dia_anterior = jugo_dia_anterior[((stats_equipos['local_team'] == stats_equipos.loc[actual,'visitor_team'])|(stats_equipos['visitor_team'] == stats_equipos.loc[actual,'visitor_team']))]
    local_jugo_dia_anterior = jugo_dia_anterior[((stats_equipos['local_team'] == stats_equipos.loc[actual,'local_team'])|(stats_equipos['visitor_team'] == stats_equipos.loc[actual,'local_team']))]
    
    diez_dias_previos = stats_equipos[(stats_equipos['number_date'] > stats_equipos.loc[actual,'previous_10_days']) & (stats_equipos['number_date'] < (stats_equipos.loc[actual,'number_date']))]
    
    if not(local_jugo_dia_anterior.empty):
        stats_equipos.loc[actual,'local_played_previous_date'] = 1
   
    if not (visitor_jugo_dia_anterior.empty):
        stats_equipos.loc[actual,'visitor_played_previous_date'] = 1
    
    if not (diez_dias_previos.empty):                                 

        local_jugo_local = diez_dias_previos[((diez_dias_previos['local_team'] == stats_equipos.loc[actual,'local_team']))]        
        local_jugo_visitante = diez_dias_previos[((diez_dias_previos['visitor_team'] == stats_equipos.loc[actual,'local_team']))]   
        visitante_jugo_local = diez_dias_previos[((diez_dias_previos['local_team'] == stats_equipos.loc[actual,'visitor_team']))]
        visitante_jugo_visitante = diez_dias_previos[((diez_dias_previos['visitor_team'] == stats_equipos.loc[actual,'visitor_team']))]

        if not (local_jugo_local.empty | local_jugo_visitante.empty):
            maximo_local_jugo_local = local_jugo_local[max(local_jugo_local['number_date']) == local_jugo_local['number_date']]    
            maximo_local_jugo_visitante = local_jugo_visitante[max(local_jugo_visitante['number_date']) == local_jugo_visitante['number_date']]
            maximo_local_jugo_local = maximo_local_jugo_local.set_index('number_date')
            maximo_local_jugo_visitante = maximo_local_jugo_visitante.set_index('number_date')

            
            if (maximo_local_jugo_local.index.max() > maximo_local_jugo_visitante.index.max()):
                
                stats_equipos.loc[actual,'place_local_played'] = stats_equipos.loc[actual, 'local_team']
                stats_equipos.loc[actual, 'avg_distance_place_local_played'] = 0
                stats_equipos.loc[actual,'local_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_local_jugo_local.index.max()                
                stats_equipos.loc[actual,'local_dif_between_previous_game'] = (stats_equipos.loc[actual,'local_dif_between_previous_game'] / np.timedelta64(1, 'D'))
                
                if(maximo_local_jugo_local.loc[maximo_local_jugo_local.index.max(), 'local_played_local'] > -1):
                    stats_equipos.loc[actual,'local_played_local'] = maximo_local_jugo_local.loc[maximo_local_jugo_local.index.max(), 'local_played_local'] + 1
                    stats_equipos.loc[actual,'local_played_visitor'] = 0
                else:
                    stats_equipos.loc[actual,'local_played_local'] = 1
                    stats_equipos.loc[actual,'local_played_visitor'] = 0

            elif (maximo_local_jugo_local.index.max() < maximo_local_jugo_visitante.index.max()):
                
                stats_equipos.loc[actual,'place_local_played'] = maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_team']
                stats_equipos.loc[actual,'local_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_local_jugo_visitante.index.max()                
                stats_equipos.loc[actual,'local_dif_between_previous_game'] = (stats_equipos.loc[actual,'local_dif_between_previous_game'] / np.timedelta64(1, 'D'))              
                
                if(maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_played_visitor'] > -1):
                    stats_equipos.loc[actual,'local_played_visitor'] = maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_played_visitor'] + 1
                    stats_equipos.loc[actual,'local_played_local'] = 0
                else:
                    stats_equipos.loc[actual,'local_played_visitor'] = 1
                    stats_equipos.loc[actual,'local_played_local'] = 0
                    
        elif not (local_jugo_local.empty):
                        
            maximo_local_jugo_local = local_jugo_local[max(local_jugo_local['number_date']) == local_jugo_local['number_date']]
            maximo_local_jugo_local = maximo_local_jugo_local.set_index('number_date')
            
            stats_equipos.loc[actual,'place_local_played'] = stats_equipos.loc[actual, 'local_team']
            stats_equipos.loc[actual, 'avg_distance_place_local_played'] = 0
            stats_equipos.loc[actual,'local_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_local_jugo_local.index.max()
            stats_equipos.loc[actual,'local_dif_between_previous_game'] = (stats_equipos.loc[actual,'local_dif_between_previous_game'] / np.timedelta64(1, 'D'))         
            
            if(maximo_local_jugo_local.loc[maximo_local_jugo_local.index.max(), 'local_played_local'] > -1):
                stats_equipos.loc[actual,'local_played_local'] = maximo_local_jugo_local.loc[maximo_local_jugo_local.index.max(), 'local_played_local'] + 1
                stats_equipos.loc[actual,'local_played_visitor'] = 0
            else:
                stats_equipos.loc[actual,'local_played_local'] = 1
                stats_equipos.loc[actual,'local_played_visitor'] = 0
        
        elif not (local_jugo_visitante.empty):

            maximo_local_jugo_visitante = local_jugo_visitante[max(local_jugo_visitante['number_date']) == local_jugo_visitante['number_date']]
            maximo_local_jugo_visitante = maximo_local_jugo_visitante.set_index('number_date')
            
            stats_equipos.loc[actual,'place_local_played'] = maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_team']
            stats_equipos.loc[actual,'local_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_local_jugo_visitante.index.max()        
            stats_equipos.loc[actual,'local_dif_between_previous_game'] = (stats_equipos.loc[actual,'local_dif_between_previous_game'] / np.timedelta64(1, 'D'))
            
            if(maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_played_visitor'] > -1):
                stats_equipos.loc[actual,'local_played_visitor'] = maximo_local_jugo_visitante.loc[maximo_local_jugo_visitante.index.max(), 'local_played_visitor'] + 1
                stats_equipos.loc[actual,'local_played_local'] = 0
            else:
                stats_equipos.loc[actual,'local_played_visitor'] = 1
                stats_equipos.loc[actual,'local_played_local'] = 0            
        
        if not (visitante_jugo_local.empty | visitante_jugo_visitante.empty):
                
            maximo_visitante_jugo_local = visitante_jugo_local[max(visitante_jugo_local['number_date']) == visitante_jugo_local['number_date']]
            maximo_visitante_jugo_visitante = visitante_jugo_visitante[max(visitante_jugo_visitante['number_date']) == visitante_jugo_visitante['number_date']]
            maximo_visitante_jugo_local = maximo_visitante_jugo_local.set_index('number_date')
            maximo_visitante_jugo_visitante = maximo_visitante_jugo_visitante.set_index('number_date')

            if (maximo_visitante_jugo_local.index.max() > maximo_visitante_jugo_visitante.index.max()):
                
                stats_equipos.loc[actual,'place_visitor_played'] = maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(),'local_team']
                stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = stats_equipos.loc[actual,'number_date'] - maximo_visitante_jugo_local.index.max()                
                stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = (stats_equipos.loc[actual,'visitor_dif_between_previous_game'] / np.timedelta64(1, 'D'))     
                
                if(maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(), 'visitor_played_local'] > -1):
                    stats_equipos.loc[actual,'visitor_played_local'] = maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(),'visitor_played_local'] + 1
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 0
                else:
                    stats_equipos.loc[actual,'visitor_played_local'] = 1
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 0

            elif (maximo_visitante_jugo_local.index.max() < maximo_visitante_jugo_visitante.index.max()):
                
                stats_equipos.loc[actual,'place_visitor_played'] = maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'local_team']
                stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_visitante_jugo_visitante.index.max()                
                stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = (stats_equipos.loc[actual,'visitor_dif_between_previous_game'] / np.timedelta64(1, 'D'))      
                
                if(maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'visitor_played_visitor'] > -1):
                    stats_equipos.loc[actual,'visitor_played_visitor'] = maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'visitor_played_visitor'] + 1
                    stats_equipos.loc[actual,'visitor_played_local'] = 0
                else:
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 1
                    stats_equipos.loc[actual,'visitor_played_local'] = 0
                    
        elif not (visitante_jugo_local.empty):            
            
            maximo_visitante_jugo_local = visitante_jugo_local[max(visitante_jugo_local['number_date']) == visitante_jugo_local['number_date']]
            maximo_visitante_jugo_local = maximo_visitante_jugo_local.set_index('number_date')
            
            stats_equipos.loc[actual,'place_visitor_played'] = maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(), 'local_team']
            stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_visitante_jugo_local.index.max()                
            stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = (stats_equipos.loc[actual,'visitor_dif_between_previous_game'] / np.timedelta64(1, 'D'))    
            
            if(maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(), 'visitor_played_local'] > -1):
                    
                    stats_equipos.loc[actual,'visitor_played_local'] = maximo_visitante_jugo_local.loc[maximo_visitante_jugo_local.index.max(), 'visitor_played_local'] + 1
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 0
            else:
                    stats_equipos.loc[actual,'visitor_played_local'] = 1
                    stats_equipos.loc[actual,'visitor_played_visitor'] = 0
                    
        elif not (visitante_jugo_visitante.empty):
            
            maximo_visitante_jugo_visitante = visitante_jugo_visitante[max(visitante_jugo_visitante['number_date']) == visitante_jugo_visitante['number_date']]
            maximo_visitante_jugo_visitante = maximo_visitante_jugo_visitante.set_index('number_date')
            
            stats_equipos.loc[actual,'place_visitor_played'] = maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'local_team']
            stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = stats_equipos.loc[actual, 'number_date'] - maximo_visitante_jugo_visitante.index.max()                
            stats_equipos.loc[actual,'visitor_dif_between_previous_game'] = (stats_equipos.loc[actual,'visitor_dif_between_previous_game'] / np.timedelta64(1, 'D'))
            
            if(maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'visitor_played_visitor'] > -1):
                stats_equipos.loc[actual,'visitor_played_visitor'] = maximo_visitante_jugo_visitante.loc[maximo_visitante_jugo_visitante.index.max(), 'visitor_played_visitor'] + 1
                stats_equipos.loc[actual,'visitor_played_local'] = 0
            else:
                stats_equipos.loc[actual,'visitor_played_visitor'] = 1
                stats_equipos.loc[actual,'visitor_played_local'] = 0

C:\Users\34680\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\34680\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [8]:
distancias.rename(columns={'Equipo A':'local_team','Equipo B':'visitor_team'}, inplace=True)
stats_equipos = pd.merge(stats_equipos, distancias, on=['local_team', 'visitor_team'], how='left')
stats_equipos.rename(columns={'Distancia':'distance_between_stadiums'}, inplace=True)

In [9]:
distancias.rename(columns={'visitor_team':'place_local_played'}, inplace=True)
stats_equipos = pd.merge(stats_equipos, distancias, on=['local_team', 'place_local_played'], how='left')
stats_equipos.rename(columns={'Distancia':'distance_local_traveled'}, inplace=True)

In [10]:
distancias.rename(columns={'place_local_played':'place_visitor_played'}, inplace=True)
stats_equipos = pd.merge(stats_equipos, distancias, on=['local_team', 'place_visitor_played'], how='left')
stats_equipos.rename(columns={'Distancia':'distance_visitor_traveled'}, inplace=True)

In [11]:
for actual in stats_equipos.index:
    if (stats_equipos.loc[actual, 'avg_distance_place_local_played'] == 0):
        stats_equipos.loc[actual, 'distance_local_traveled'] = 0
    if (stats_equipos.loc[actual, 'distance_local_traveled'] != 'NaN'):
        stats_equipos.loc[actual, 'avg_distance_place_local_played'] = stats_equipos.loc[actual, 'distance_local_traveled'] / stats_equipos.loc[actual,'local_dif_between_previous_game']
    if (stats_equipos.loc[actual, 'distance_visitor_traveled'] != 'NaN'):
        stats_equipos.loc[actual, 'avg_distance_place_visitor_played'] = stats_equipos.loc[actual, 'distance_visitor_traveled'] / stats_equipos.loc[actual,'visitor_dif_between_previous_game']


In [ ]:
stats_equipos = stats_equipos.drop(columns =["previous_10_days", 'previous_date'], inplace = True) 

In [ ]:
pd.options.display.max_rows = 30
pd.options.display.max_columns = 300

In [12]:
stats_equipos.to_csv('./input_extendido.csv')